In [180]:
import random
import pandas as pd 
import numpy as np


In [181]:
# load data
df = pd.read_csv('sample.csv')


print(df.head())

   Unnamed: 0 ticker      risk    return
0           0    LMB  0.665555  2.459677
1           1  GECCM  0.011733  0.202107
2           2   OXBR  0.810532  2.040541
3           3   LIVE  0.630844  3.729542
4           4   NVEE  0.330513  1.610970


* Gene: A gene in this case will represent the proportion of capital invested in a specific stock. Each gene will be a floating-point number between 0 and 1, where the sum of all genes in a chromosome will always equal 1.

* Chromosome: A chromosome will represent a proposed solution to the problem of maximizing return while minimizing risk. Each chromosome will be a list of genes representing the proportion of capital invested in each stock.

* Population: The population will be a list of chromosomes representing multiple proposed solutions to the problem. Each chromosome in the population will be randomly generated initially, and then evolved over multiple generations using genetic operators such as selection, crossover, and mutation.

In [182]:

# Define the number of genes (stocks) and chromosomes (solutions) in the population
numGenes = 400
numChromosomes = 100
# define the hyperparameters for the mutation and crossover functions
mutation_rate = 0.1
crossover_rate = 0.8

# Define the gene as a floating-point number between 0 and 1
def createGene():
    gene = round(random.uniform(0, 1), 2)
#    print(f"Gene created: {gene}")
    return gene



# Define the chromosome as a list of genes where the sum of all genes equals 1
def createChromosome():
    chromosome = []
    sumGenes = 0
    for i in range(numGenes):
        gene = createGene()
        chromosome.append(gene)
        sumGenes += gene
    # Normalize the genes so that the sum equals 1
    for i in range(numGenes):
        chromosome[i] /= sumGenes
#    print(f"Chromosome created: {chromosome}")
    return chromosome

# Create an initial population of chromosomes randomly
def createPopulation():
    population = []
    for i in range(numChromosomes):
        chromosome = createChromosome()
        population.append(chromosome)
#    print(f"Population created: {population}")
    return population

# define fitness function
def fitness(chromosome):
    # calculate return and risk
    total_return = 0
    total_risk = 0
    for i in range(numGenes):
        total_return += chromosome[i] * df['return'][i]
        total_risk += chromosome[i] * df['risk'][i]
    # calculate number of shares
    num_shares = sum([1 for i in range(numGenes) if chromosome[i] > 0])
    # combine into fitness score
    return total_return / total_risk * num_shares, num_shares


# Perform tournament selection to select the best parent chromosome
def tournamentSelection(population):
    # randomly select a subset of chromosomes from the population
    tournament = random.sample(population, k=5)
    # sort the tournament by fitness in descending order
    tournament.sort(key=lambda chromosome: fitness(chromosome)[0], reverse=True)
    # return the best chromosome as the parent
    parent = tournament[0]
#    print(f"Parent selected: {parent}")
    return parent

# Perform single point crossover to create two offspring chromosomes
def singlePointCrossover(parent1, parent2):
    # randomly select a crossover point
    crossover_point = random.randint(0, numGenes-1)
    # create offspring chromosomes by swapping genes from the parents
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
#    print(f"Parents: {parent1}, {parent2}")
#    print(f"Offspring created: {offspring1}, {offspring2}")
    return offspring1, offspring2


# Perform crossover on the current population
def crossover(population):
    new_population = []
    for i in range(numChromosomes // 2):
        # select two parent chromosomes using tournament selection
        parent1 = tournamentSelection(population)
        parent2 = tournamentSelection(population)
        # perform crossover to create two offspring chromosomes
        offspring1, offspring2 = singlePointCrossover(parent1, parent2)
        # append the offspring chromosomes to the new population
        new_population.append(offspring1)
        new_population.append(offspring2)
#        print(f"Offspring created: {offspring1}, {offspring2}")
    return new_population

# Define the mutation function
def mutate(chromosome):
    for gene_index in range(numGenes):
        if random.random() < mutation_rate:
            # randomly generate a new gene
            new_gene = createGene()
            # replace the old gene with the new gene
            chromosome[gene_index] = new_gene
    return chromosome


def geneticAlgorithm():
    # create initial population
    population = createPopulation()
    
    # iterate through generations
    for generation in range(10):
        print(f"Generation: {generation}")
        # perform crossover on the population
        new_population = crossover(population)
        
        # mutate the new population
        for i in range(len(new_population)):
            new_population[i] = mutate(new_population[i])

                    
        # evaluate fitness of the new population
        fitness_scores = []
        for chromosome in new_population:
            fitness_score = fitness(chromosome)
            fitness_scores.append(fitness_score)
        
        # sort the new population by fitness in descending order
        new_population = [chromosome for _,chromosome in sorted(zip(fitness_scores,new_population), reverse=True)]
        
        # select the top 100 chromosomes to be the next generation
        population = new_population[:numChromosomes]
        
    # return the best chromosome from the final generation
    best_chromosome = max(population, key=lambda chromosome: fitness(chromosome)[0])
    print(f"Best chromosome: {best_chromosome}")
    return best_chromosome




# create initial population
population = createPopulation()
# perform crossover on the population
new_population = crossover(population)
# print the new population
#print(f"New population: {new_population}")


createPopulation()
chromosome = createChromosome()
# perform crossover on the population
new_population = crossover(population)
#print("Chromosome: ", chromosome)
#print("Fitness: ", fitness(chromosome))
# print the new population
#print(f"New population: {new_population}")

solution = geneticAlgorithm() 
solution[:] = [x / 100 for x in solution]
print(solution)

# Read the CSV file into a DataFrame
df = pd.read_csv('sample.csv')
df['value'] = solution 
df.to_csv('updated_sample1.csv', index=False)


Generation: 0
Generation: 1
Generation: 2
Generation: 3
Generation: 4
Generation: 5
Generation: 6
Generation: 7
Generation: 8
Generation: 9
Best chromosome: [0.83, 0.0009143089348300907, 0.004114390206735409, 0.42, 0.68, 0.05, 0.87, 0.9, 0.81, 0.42, 0.74, 0.0025397470411946967, 0.0018794128104840755, 0.01, 0.43, 0.16, 0.004774724437446029, 0.45, 0.003606440798496469, 0.002336567277899121, 0.0006095392898867271, 0.16, 0.77, 0.09, 0.62, 0.29, 0.46, 0.14, 0.19, 0.26, 0.0014222583430690302, 0.05, 0.0032508762127292116, 0.33, 0.22, 0.0022335831637464642, 0.5, 0.33, 0.24, 0.00039708145133270475, 0.43, 0.75, 0.4, 0.18, 0.0039211793319104595, 0.34, 4.963518141658809e-05, 0.8, 0.004020449694743636, 0.08, 0.0033751923363279906, 0.000496351814165881, 0.09, 0.0033751923363279906, 0.57, 0.9, 0.41, 0.004268625601826576, 0.25, 0.0012905147168312905, 0.38, 0.06, 0.43, 0.17, 0.0014890554424976427, 0.003176651610661638, 0.06, 0.0013897850796644668, 0.19, 0.001985407256663524, 0.004616071871742693, 0.002